<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6a7005eaa595ea2b38117b4f0d458cdd07a6191d4dbbe46ddccbcf8f110fc985
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.3
    Uninstalling protobuf-6.33.3:
      Successfully uninstalled protobuf-6.33.3


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-12 12:53:54
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: -52.01 K (-0.37%)
Current PnL: -28.55 L (-18.21%)
CY Booked + Current PnL: -14.26 L (-9.09%)
-------------------
Total profit:  1.79 L
Total loss:  -30.34 L
-------------------
Total Booked + Current PnL: 12.72 L (9.8%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.17%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.01 L (66.89%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.29%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.76,15.40,5.19,21.40,15897.0,40880.0,306306.0,7.09,63.0,X-LC,5.89,14.0,2.57,2.18,22.70,XY25,NTT,FMCG
17,COALINDIA,484.83,3.06,6.03,12.45,19.23,19861.0,9066.0,159526.0,24.27,72.0,L-LC,7.66,182.0,0.46,1.14,28.58,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-2.02,-22.30,28.82,0.09,22568.0,-22472.0,78305.0,81.64,34.0,M-SC,3.34,253.0,-1.00,0.56,2.88,OX40N,NTT,DURABLES
49,MASFIN,398.61,0.52,-4.41,27.68,22.05,25925.0,-4320.0,93660.0,-17.80,46.0,H-SC,6.26,168.0,-0.17,0.67,36.14,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.53,-15.19,17.93,0.02,26544.0,-26515.0,148044.0,126.00,51.0,M-SC,8.30,240.0,-1.00,1.05,28.77,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,3.06,6.03,12.45,19.23,19861.0,9066.0,159526.0,24.27,72.0,L-LC,7.66,182.0,0.46,1.14,28.58,XY25,ATH,MINING
37,IEX,219.00,2.82,-3.45,53.94,48.63,104130.0,-6907.0,193047.0,-35.77,52.0,H-SC,13.74,137.0,-0.07,1.37,7.76,XR,NTT,MISC
40,INDUSINDBK,1800.00,2.29,-33.37,99.45,32.89,51159.0,-25764.0,51442.0,-735.55,63.0,L-MC,7.90,259.0,-0.50,0.37,41.67,XR,NTT,BANKS
5,ASIANPAINT,3460.25,2.16,0.89,19.88,20.95,50496.0,2235.0,254003.0,-0.07,62.0,X-LC,12.42,36.0,0.04,1.81,36.84,X40,ATH,PAINTS
57,RECLTD,446.00,1.79,0.19,20.51,20.74,41749.0,385.0,203555.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.45,10.82,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJESHEXPO,518.00,-4.02,-68.72,219.95,0.07,94367.0,-94273.0,42904.0,1471.97,31.0,L-SC,2.05,269.0,-1.00,0.31,6.70,OX40N,NTT,JEWELLERY
65,SIEMENS,4671.50,-3.12,-20.77,58.43,25.51,86146.0,-38660.0,147435.0,-9.42,34.0,H-LC,1.98,50.0,-0.45,1.05,8.65,AR,ATH,ELECTRICAL
76,TRIDENT,48.00,-2.98,-31.97,88.90,28.51,55796.0,-29492.0,62763.0,-19.62,32.0,M-SC,7.77,226.0,-0.53,0.45,7.40,XR,NTT,TEXTILES
60,REPCOHOME,880.00,-2.92,-12.42,116.75,89.82,292935.0,-35597.0,250908.0,-56.04,46.0,H-SC,4.22,161.0,-0.12,1.79,32.75,XY24,NTT,FINANCE
9,BAJAJHLDNG,14451.52,-2.38,-3.61,34.06,29.22,62418.0,-6865.0,183260.0,-6.00,37.0,X-LC,11.74,29.0,-0.11,1.30,5.24,X40,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,1.79,0.19,20.51,20.74,41749.0,385.0,203555.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.45,10.82,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.96,-4.30,117.27,107.91,165688.0,-6356.0,141288.0,-22.17,45.0,M-SC,10.11,220.0,-0.04,1.01,5.35,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.53,-15.19,17.93,0.02,26544.0,-26515.0,148044.0,126.00,51.0,M-SC,8.30,240.0,-1.00,1.05,28.77,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.19,-18.14,61.29,32.03,66144.0,-23915.0,107920.0,-11.34,64.0,H-SC,6.82,157.0,-0.36,0.77,20.88,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-2.02,-22.30,28.82,0.09,22568.0,-22472.0,78305.0,81.64,34.0,M-SC,3.34,253.0,-1.00,0.56,2.88,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.71,-22.97,47.31,13.47,98272.0,-61947.0,207720.0,-68.31,45.0,H-SC,4.52,159.0,-0.63,1.48,5.62,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,1.79,0.19,20.51,20.74,41749.0,385.0,203555.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.45,10.82,XY25,NTT,FINANCE
17,COALINDIA,484.83,3.06,6.03,12.45,19.23,19861.0,9066.0,159526.0,24.27,72.0,L-LC,7.66,182.0,0.46,1.14,28.58,XY25,ATH,MINING
84,WIPRO,420.00,0.74,8.24,59.15,72.27,108487.0,13962.0,183410.0,-6.99,52.0,M-LC,6.21,101.0,0.13,1.31,14.92,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.74,8.24,59.15,72.27,108487.0,13962.0,183410.0,-6.99,52.0,M-LC,6.21,101.0,0.13,1.31,14.92,XR,NTT,IT
25,FINCABLES,1641.55,-1.96,-4.30,117.27,107.91,165688.0,-6356.0,141288.0,-22.17,45.0,M-SC,10.11,220.0,-0.04,1.01,5.35,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.24,-8.03,124.77,106.72,141528.0,-9905.0,113431.0,-54.44,34.0,H-SC,7.66,138.0,-0.07,0.81,13.47,AR,ATH,MISC
37,IEX,219.00,2.82,-3.45,53.94,48.63,104130.0,-6907.0,193047.0,-35.77,52.0,H-SC,13.74,137.0,-0.07,1.37,7.76,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.42,-6.03,44.33,35.63,87305.0,-12630.0,196944.0,-19.40,34.0,H-MC,3.35,119.0,-0.14,1.40,9.64,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.40,-17.04,33.53,10.78,66851.0,-40942.0,199376.0,-50.55,14.0,X-LC,7.09,1.0,-0.61,1.42,0.40,X40,NTT,FMCG
7,AWL,485.00,-1.26,-28.64,120.29,57.19,279926.0,-93418.0,232709.0,-66.31,21.0,X-MC,9.83,58.0,-0.33,1.66,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,0.10,-13.15,50.39,30.62,103748.0,-31164.0,205890.0,-38.32,26.0,X-MC,16.66,57.0,-0.30,1.47,0.10,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.81,-41.34,130.48,35.20,98486.0,-53190.0,75480.0,-1.08,27.0,X-SC,17.48,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
78,UNITDSPR,1644.00,-0.66,-2.87,24.34,20.77,55354.0,-6726.0,227418.0,-11.38,29.0,X-MC,1.32,70.0,-0.12,1.62,3.85,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.66,-2.87,24.34,20.77,55354.0,-6726.0,227418.0,-11.38,29.0,X-MC,1.32,70.0,-0.12,1.62,3.85,X40N,NTT,BREWERIES
45,JIOFIN,387.00,0.03,-7.57,34.68,24.48,85702.0,-20244.0,247121.0,-9.13,38.0,X-LC,1.54,37.0,-0.24,1.76,43.24,XY24,BTT,FINANCE
65,SIEMENS,4671.50,-3.12,-20.77,58.43,25.51,86146.0,-38660.0,147435.0,-9.42,34.0,H-LC,1.98,50.0,-0.45,1.05,8.65,AR,ATH,ELECTRICAL
81,VBL,671.64,0.74,-0.49,36.35,35.67,114240.0,-1563.0,314279.0,-12.33,55.0,X-LC,2.28,5.0,-0.01,2.24,13.29,X40N,ATH,FMCG
73,TCS,4389.96,0.40,-11.44,36.30,20.71,121588.0,-43284.0,334953.0,-23.62,49.0,X-LC,2.34,3.0,-0.36,2.38,11.92,X40,ATH,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.25,-28.60,129.90,64.15,220700.0,-68051.0,169900.0,-58.06,32.0,X-SC,6.40,84.0,-0.31,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-1.26,-28.64,120.29,57.19,279926.0,-93418.0,232709.0,-66.31,21.0,X-MC,9.83,58.0,-0.33,1.66,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.81,-41.34,130.48,35.20,98486.0,-53190.0,75480.0,-1.08,27.0,X-SC,17.48,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.53,-6.62,43.70,34.19,87134.0,-14128.0,199392.0,-34.39,33.0,X-MC,4.07,60.0,-0.16,1.42,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-1.94,-49.16,199.58,52.31,147288.0,-71361.0,73799.0,-45.89,34.0,X-SC,7.95,92.0,-0.48,0.53,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.57,-28.61,100.08,42.84,46447.0,-18596.0,46410.0,-53.17,48.0,X-SC,26.52,83.0,-0.40,0.33,5.00,XY24,NTT,MISC
58,RELAXO,1176.00,-1.94,-49.16,199.58,52.31,147288.0,-71361.0,73799.0,-45.89,34.0,X-SC,7.95,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.81,-41.34,130.48,35.20,98486.0,-53190.0,75480.0,-1.08,27.0,X-SC,17.48,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,1.23,-0.65,24.88,24.08,31381.0,-821.0,126129.0,-8.70,49.0,X-SC,6.15,91.0,-0.03,0.90,18.84,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.63,-18.24,75.30,43.32,100269.0,-29712.0,133160.0,-29.79,41.0,X-SC,7.95,90.0,-0.30,0.95,2.24,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.40,-17.04,33.53,10.78,66851.0,-40942.0,199376.0,-50.55,14.0,X-LC,7.09,1.0,-0.61,1.42,0.40,X40,NTT,FMCG
75,TMPV,600.00,-0.96,-28.11,71.06,22.98,115399.0,-63501.0,162397.0,-25.52,38.0,X-LC,4.10,2.0,-0.55,1.16,2.14,XY24,NTT,AUTO
73,TCS,4389.96,0.40,-11.44,36.30,20.71,121588.0,-43284.0,334953.0,-23.62,49.0,X-LC,2.34,3.0,-0.36,2.38,11.92,X40,ATH,IT
81,VBL,671.64,0.74,-0.49,36.35,35.67,114240.0,-1563.0,314279.0,-12.33,55.0,X-LC,2.28,5.0,-0.01,2.24,13.29,X40N,ATH,FMCG
41,INFY,1972.00,-0.31,9.54,22.55,34.24,77287.0,29850.0,342738.0,-14.57,49.0,X-LC,7.68,6.0,0.39,2.44,18.61,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.29,-33.37,99.45,32.89,51159.0,-25764.0,51442.0,-735.55,63.0,L-MC,7.90,259.0,-0.50,0.37,41.67,XR,NTT,BANKS
69,SURYODAY,216.00,-0.48,-19.18,52.23,23.03,75591.0,-34343.0,144728.0,57.89,52.0,H-SC,3.95,165.0,-0.45,1.03,43.60,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.53,-15.19,17.93,0.02,26544.0,-26515.0,148044.0,126.00,51.0,M-SC,8.30,240.0,-1.00,1.05,28.77,OX40N,NTT,PAINTS
17,COALINDIA,484.83,3.06,6.03,12.45,19.23,19861.0,9066.0,159526.0,24.27,72.0,L-LC,7.66,182.0,0.46,1.14,28.58,XY25,ATH,MINING
84,WIPRO,420.00,0.74,8.24,59.15,72.27,108487.0,13962.0,183410.0,-6.99,52.0,M-LC,6.21,101.0,0.13,1.31,14.92,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.49,-31.26,64.24,12.90,96487.0,-68299.0,150197.0,121.15,57.0,H-SC,10.03,173.0,-0.71,1.07,24.89,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,-0.19,-18.14,61.29,32.03,66144.0,-23915.0,107920.0,-11.34,64.0,H-SC,6.82,157.0,-0.36,0.77,20.88,OX40N,NTT,IT
17,COALINDIA,484.83,3.06,6.03,12.45,19.23,19861.0,9066.0,159526.0,24.27,72.0,L-LC,7.66,182.0,0.46,1.14,28.58,XY25,ATH,MINING
57,RECLTD,446.00,1.79,0.19,20.51,20.74,41749.0,385.0,203555.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.45,10.82,XY25,NTT,FINANCE
56,RAJOOENG,143.10,0.10,-33.63,110.35,39.61,75071.0,-34470.0,68030.0,-55.07,49.0,H-SC,30.28,136.0,-0.46,0.48,10.12,AR,ATH,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.43
1,25,44.53
2,50,76.62


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.53
MC    30.81
LC    26.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.31
X40      24.04
X40N     12.81
XY25     10.86
XR        8.98
AR        8.50
OX40N     7.61
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.40
X-LC    23.22
H-SC    22.75
M-SC    11.19
X-SC     7.81
H-MC     4.59
M-MC     1.45
M-LC     1.31
L-LC     1.14
H-LC     1.05
L-SC     0.78
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.20,-9.02,44.79
IT,12.92,-17.78,77.18
FINANCE,11.06,-14.87,63.08
MISC,7.05,-31.06,84.68
ELECTRICAL,5.88,-13.66,55.29
PAINTS,5.82,-12.99,29.77
INSURANCE,4.75,-1.65,36.44
PHARMA,3.94,-5.98,39.34
AUTO,2.78,-34.49,81.12


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3277971.0,21
AR,1376187.0,10
XR,1231537.0,12
X40,1142168.0,15
X40N,944533.0,9
OX40N,723895.0,10
XY25,416492.0,7
SR,287690.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3599238.0,24
X-MC,1605829.0,16
M-SC,1486625.0,15
X-LC,961392.0,13
X-SC,823941.0,8
H-MC,433627.0,3
L-SC,182419.0,2
M-LC,108487.0,1
H-LC,86146.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1270964.0      6
           AR         929396.0      5
M-SC       XY24       851062.0      6
H-SC       XR         795687.0      7
X-MC       X40        522491.0      7
           XY24       449950.0      3
X-LC       X40        419766.0      6
X-MC       X40N       409850.0      4
X-SC       X40N       325502.0      3
H-SC       OX40N      315501.0      4
M-SC       OX40N      314027.0      5
X-SC       XY24       298528.0      3
H-SC       SR         287690.0      2
H-MC       AR         227261.0      2
X-MC       XY25       223538.0      2
X-LC       X40N       209181.0      2
H-MC       XY24       206366.0      1
X-LC       XY24       201101.0      2
X-SC       X40        199911.0      2
M-SC       XR         188152.0      2
           AR         133384.0      2
X-LC       XY25       131344.0      3
M-LC       XR         108487.0      1
L-SC       OX40N       94367.0      1
           XR          88052.0      1
H-LC       AR          86146.0      1
L-MC       XR          51159.0      1
M-MC       XY25        41749.0      1
L-LC       XY25        19861.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
